In [1]:
"""
https://leetcode.com/problems/design-bounded-blocking-queue/
https://leetcode.ca/2019-03-02-1188-Design-Bounded-Blocking-Queue/


Implement a thread-safe bounded blocking queue that has the following methods:

BoundedBlockingQueue(int capacity) The constructor initializes the queue with a maximum capacity.
void enqueue(int element) Adds an element to the front of the queue. 
If the queue is full, the calling thread is blocked until the queue is no longer full.
int dequeue() Returns the element at the rear of the queue and removes it. 
If the queue is empty, the calling thread is blocked until the queue is no longer empty.
int size() Returns the number of elements currently in the queue.
Your implementation will be tested using multiple threads at the same time. 
Each thread will either be a producer thread that only makes calls to the enqueue method 
or a consumer thread that only makes calls to the dequeue method. 
The size method will be called after every test case.

Please do not use built-in implementations of bounded blocking queue as this will not be accepted in an interview.


Constraints:

1 <= Number of Prdoucers <= 8
1 <= Number of Consumers <= 8
1 <= size <= 30
0 <= element <= 20
The number of calls to enqueue is greater than or equal to the number of calls to dequeue.
At most 40 calls will be made to enque, deque, and size.

"""

import threading
import time
from collections import deque

class BoundedBlockingQueue:
    def __init__(self, capacity):
        self.capacity = capacity
        self.queue = deque()
        self.producerSemaphore = threading.Semaphore(self.capacity)
        self.consumerSemaphore = threading.Semaphore(0)

    def enqueue(self, val):
        self.producerSemaphore.acquire()
        print("Produced", val)
        self.queue.append(val)
        self.consumerSemaphore.release()

    def dequeue(self):
        self.consumerSemaphore.acquire()
        val = self.queue.popleft()
        print("Consumed", val)
        self.producerSemaphore.release()
        return val

    def size(self):
        return len(self.queue)

# 1
# 1
# ["BoundedBlockingQueue","enqueue","dequeue","dequeue","enqueue","enqueue","enqueue","enqueue","dequeue"]
# [[2],[1],[],[],[0],[2],[3],[4],[]]
# Output:
# [1,0,2,2]
# Number of producer threads = 1
# Number of consumer threads = 1

queue = BoundedBlockingQueue(2)

outputs = []
lock = threading.Lock()
def consume(val):
    lock.acquire()
    outputs.append(val)
    lock.release()

def producer1():
    print("producer1 started")
    # time 0
    queue.enqueue(1)
    # time 3
    time.sleep(3)
    queue.enqueue(0)
    # time 4
    time.sleep(1)
    queue.enqueue(2)
    # time 5
    time.sleep(1)
    queue.enqueue(3)
    # time 6
    time.sleep(1)
    queue.enqueue(4)

def consumer1():
    print("consumer1 started")
    # time 1
    time.sleep(1)
    consume(queue.dequeue())
    # time 2
    time.sleep(1)
    consume(queue.dequeue())
    # time 7
    time.sleep(5)
    consume(queue.dequeue())


threads = []
threads.append(threading.Thread(target=producer1, args=[]))
threads.append(threading.Thread(target=consumer1, args=[]))
for t in threads:
    t.start()
for t in threads:
    t.join()
print("queue size", queue.size())
print("output", outputs)
    


# # 3
# # 4
# # ["BoundedBlockingQueue","enqueue","enqueue","enqueue","dequeue","dequeue","dequeue","enqueue"]
# # [[3],[1],[0],[2],[],[],[],[3]]
# # Output:
# # [1,0,2,1]
# # Number of producer threads = 3
# # Number of consumer threads = 4

queue = BoundedBlockingQueue(3)
outputs = []

def producer1():
    # time 0
    queue.enqueue(1)
    # time 6
    time.sleep(6)
    queue.enqueue(4)

def producer2():
    # time 1
    time.sleep(1)
    queue.enqueue(0)

def producer3():
    # time 2
    time.sleep(2)
    queue.enqueue(2)

def consumer1():
    # time 3
    time.sleep(3)
    consume(queue.dequeue())

def consumer2():
    # time 4
    time.sleep(4)
    consume(queue.dequeue())

def consumer3():
    # time 5
    time.sleep(5)
    consume(queue.dequeue())

threads = []
threads.append(threading.Thread(target=producer1, args=[]))
threads.append(threading.Thread(target=producer2, args=[]))
threads.append(threading.Thread(target=producer3, args=[]))
threads.append(threading.Thread(target=consumer1, args=[]))
threads.append(threading.Thread(target=consumer2, args=[]))
threads.append(threading.Thread(target=consumer3, args=[]))
for t in threads:
    t.start()
for t in threads:
    t.join()
print("queue size", queue.size()) # 1
print("output", outputs) # 3



producer1 started
Produced 1
consumer1 started
Consumed 1
Produced 0
Consumed 0
Produced 2
Produced 3
Consumed 2
Produced 4
queue size 2
output [1, 0, 2]
Produced 1
Produced 0
Produced 2
Consumed 1
Consumed 0
Consumed 2
Produced 4
queue size 1
output [1, 0, 2]
